In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv', lineterminator='\n' )
test = pd.read_csv('test.csv', lineterminator='\n')

In [3]:
test.dropna(inplace=True)

In [4]:
test.reset_index(inplace=True)
test = test[['summary', 'article']]

In [5]:
test.head()

,summary,article
0,पाकिस्तानी जासूस ने खुद को सैन्य कमांडर बताया ...,जम्मू कश्मीर पुलिस के एक अधिकारी ने घाटी में प...
1,समाजवादी पार्टी के पूर्व महासचिव और राज्यसभा स...,नोट के बदले वोट मामले में आरोपी समाजवादी पार्ट...
2,सैलरी के बारे में बात करते वक्‍त पर्सनल बातें ...,"VIDEO: अब सैलरी होगी कम, सेविंग ज़्यादा!"
3,आईआईटी कानपुर द्वारा तैयार सिमरन (सैटेलाइट इमे...,रेल नेविगेशन से संबंधित आईआईटी कानपुर की महत्व...
4,शिवपाल की यह टिप्पणी फिल्म 'कालिया' के डायलाग ...,प्रस्तावित समाजवादी धर्मनिरपेक्ष मोर्चा का नेत...


In [6]:
from transformers import MBartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import AlbertTokenizer, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART", do_lower_case=False, use_fast=False, keep_accents=True)


model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")

In [8]:
summaries = []
row_id = []
for i in range(50):
    try:
        inp = tokenizer(test.iloc[i,1], add_special_tokens=False, return_tensors="pt", padding=True).input_ids # tensor([[  466,  1981,    80, 25573, 64001, 64004]])

        out = tokenizer(test.iloc[i,0], add_special_tokens=False, return_tensors="pt", padding=True).input_ids 
        model_outputs=model(input_ids=inp, decoder_input_ids=out[:,0:-1], labels=out[:,1:])
        model_output=model.generate(inp, use_cache=True, num_beams=4, max_length=20, min_length=1, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>"))



        decoded_output=tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        summaries.append(decoded_output)
        row_id.append(i)
    except:
        continue

In [9]:
articles = [test.iloc[i, 1] for i in row_id]
target_summaries = [test.iloc[i, 0] for i in row_id]

In [13]:
columns = {'articles':articles, 'target_summaries':target_summaries, 'predicted_summaries': summaries}

In [14]:
df = pd.DataFrame(columns)

In [20]:
df.to_csv('summaries.csv')